# Discharge Prediction with HBV Distributed Model (Hapi)

This notebook demonstrates how to implement and run the distributed HBV hydrological model for discharge prediction using the Hapi (Hydrological library for Python) framework.

The HBV model is a conceptual hydrological model developed by the Swedish Meteorological and Hydrological Institute (SMHI) that simulates catchment runoff by accounting for processes such as snow accumulation/melting, soil moisture, and runoff response. In this notebook, we use a distributed version that applies the model at the grid cell level.

## Workflow Overview
1. **Setup & Dependencies**
2. **Data Preparation**
   - Load digital elevation model (DEM)
   - Prepare flow direction & accumulation data
   - Load meteorological forcings (precipitation, temperature, evapotranspiration)
   - Set up parameters
3. **Model Configuration**
4. **Model Execution**
5. **Results Analysis & Visualization**
6. **Model Evaluation**

### References
- Bergström, S. (1992). The HBV model - its structure and applications. SMHI Reports RH.
- [Hapi GitHub Repository](https://github.com/Serapieum-of-alex/Hapi)

## 1. Setup & Dependencies

First, we need to install and import the necessary libraries.

In [ ]:
# Install Hapi if not already installed
# Uncomment the following line to install
# !pip install Hapi

In [ ]:
# Core dependencies
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import logging
from typing import Dict, Tuple, List, Optional, Union
from datetime import datetime

# Hapi modules
from Hapi.rrm.distrrm import DistributedRRM
from Hapi.rrm.hbv import HBV
from Hapi.raster.raster import Raster

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

## 2. Data Preparation

For the distributed HBV model, we need several input datasets:

1. Digital Elevation Model (DEM)
2. Flow direction and accumulation data
3. Meteorological data (precipitation, temperature, evapotranspiration)
4. Model parameters (can be calibrated or from literature)

In [ ]:
# Define data paths (adjust according to your setup)
data_dir = "../data"
dem_path = os.path.join(data_dir, "DEM/dem.tif")

# Path to meteorological data
prec_path = os.path.join(data_dir, "MeteoData/precipitation.nc")
temp_path = os.path.join(data_dir, "MeteoData/temperature.nc")
et_path = os.path.join(data_dir, "MeteoData/evapotranspiration.nc")

# Path to flow direction and accumulation files
flow_dir_path = os.path.join(data_dir, "Geometry/flow_direction.tif")
flow_acc_path = os.path.join(data_dir, "Geometry/flow_accumulation.tif")

# Path to observed discharge data (for validation)
observed_q_path = os.path.join(data_dir, "HydroFiles/observed_discharge.csv")

### 2.1 Load and Process DEM

The DEM is the foundation for our distributed model as it provides the topographic information needed for flow routing.

In [ ]:
def load_dem(dem_path: str) -> Tuple[np.ndarray, float, float]:
    """Load DEM and extract relevant properties.

    Args:
        dem_path: Path to DEM file

    Returns:
        Tuple containing DEM array, cell size, and no data value
    """
    try:
        # Load DEM
        dem_raster = Raster(dem_path)
        dem_array = dem_raster.read_array()
        cell_size = dem_raster.cell_size
        no_data_value = dem_raster.no_data_value

        return dem_array, cell_size, no_data_value

    except Exception as e:
        logger.error(f"Error loading DEM: {str(e)}")
        # Return dummy data for demonstration
        return np.ones((100, 100)), 0.01, -9999


# Try to load DEM
try:
    dem_array, cell_size, no_data_value = load_dem(dem_path)

    # Create catchment mask
    mask = np.where(dem_array != no_data_value, 1, 0)

    # Display DEM information
    print(f"DEM Shape: {dem_array.shape}")
    print(f"DEM Resolution: {cell_size}")
    print(f"DEM No Data Value: {no_data_value}")

    # Visualize the DEM
    plt.figure(figsize=(10, 8))
    plt.imshow(dem_array, cmap="terrain")
    plt.colorbar(label="Elevation (m)")
    plt.title("Digital Elevation Model")
    plt.show()

    # Store DEM path for model execution
    DEM = dem_path

except Exception as e:
    print(f"Error loading DEM: {str(e)}")
    print("Using dummy DEM data for demonstration.")
    # Create dummy DEM for demonstration
    dem_array = np.ones((100, 100))
    cell_size = 0.01
    no_data_value = -9999
    mask = np.ones_like(dem_array)
    DEM = "dummy_dem.tif"

### 2.2 Process Flow Direction and Accumulation Data

Flow direction and accumulation maps are essential for routing the water through the catchment.

In [ ]:
def load_flow_data(
    flow_dir_path: str, flow_acc_path: str, dem_shape: Tuple[int, int]
) -> Tuple[Dict, np.ndarray]:
    """Load flow direction and accumulation data.

    Args:
        flow_dir_path: Path to flow direction raster file
        flow_acc_path: Path to flow accumulation raster file
        dem_shape: Shape of the DEM (rows, cols)

    Returns:
        Tuple containing flow accumulation dictionary and flow accumulation plan array
    """
    try:
        # For demonstration, we'll create dummy flow accumulation data
        # In a real application, you would load your prepared data

        rows, cols = dem_shape

        # Create dummy flow accumulation dictionary
        # In practice, this would be derived from flow direction
        flow_acc = {}
        for i in range(rows):
            for j in range(cols):
                # Each cell drains to the cell below it (simplified)
                if i < rows - 1:
                    flow_acc[(i, j)] = [(i + 1, j)]

        # Create dummy flow accumulation plan
        flow_acc_plan = np.zeros(dem_shape, dtype=int)
        # Simple accumulation pattern (cells accumulate from top to bottom)
        for i in range(rows):
            flow_acc_plan[i, :] = rows - i

        logger.info("Flow direction and accumulation data prepared")
        return flow_acc, flow_acc_plan

    except Exception as e:
        logger.error(f"Error preparing flow data: {str(e)}")
        raise


# Try to load flow data
try:
    flow_acc, flow_acc_plan = load_flow_data(flow_dir_path, flow_acc_path, dem_array.shape)
    print("Flow direction and accumulation data prepared.")

    # Visualize flow accumulation
    plt.figure(figsize=(10, 8))
    plt.imshow(flow_acc_plan, cmap="Blues")
    plt.colorbar(label="Flow Accumulation")
    plt.title("Flow Accumulation Map")
    plt.show()

except Exception as e:
    print(f"Error preparing flow data: {str(e)}")
    print("Using dummy flow data for demonstration.")
    # Create minimal dummy flow data
    flow_acc = {}
    flow_acc_plan = np.ones_like(dem_array)

### 2.3 Load Meteorological Data

For the HBV model, we need precipitation, temperature, and potential evapotranspiration data as forcing variables.

In [ ]:
def load_meteorological_data(
    prec_path: str, temp_path: str, et_path: str, dem_shape: Tuple[int, int]
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Load and prepare meteorological data for the HBV model.

    Args:
        prec_path: Path to precipitation data
        temp_path: Path to temperature data
        et_path: Path to potential evapotranspiration data
        dem_shape: Shape of the DEM (rows, cols)

    Returns:
        Tuple of 3D arrays for precipitation, temperature, and evapotranspiration
        Each array has shape (rows, cols, timesteps)
    """
    try:
        # For demonstration, we'll create synthetic meteorological data
        # In a real application, you would load your data files

        rows, cols = dem_shape
        n_timesteps = 365  # One year of daily data

        # Create synthetic precipitation with seasonal pattern
        np.random.seed(42)  # For reproducibility
        sp_prec = np.zeros((rows, cols, n_timesteps))
        for t in range(n_timesteps):
            # Seasonal pattern with random noise
            season_factor = np.sin(2 * np.pi * t / 365) + 1  # 0-2 range
            daily_precip = np.random.exponential(scale=2 * season_factor, size=(rows, cols))
            # Set some days to zero (dry days)
            if np.random.random() < 0.7:  # 70% chance of rain
                sp_prec[:, :, t] = daily_precip

        # Create synthetic temperature with seasonal pattern
        sp_temp = np.zeros((rows, cols, n_timesteps))
        for t in range(n_timesteps):
            # Temperature with seasonal pattern
            base_temp = 15 + 10 * np.sin(2 * np.pi * t / 365)  # 5-25°C range
            spatial_var = np.random.normal(0, 2, size=(rows, cols))
            sp_temp[:, :, t] = base_temp + spatial_var

        # Create synthetic evapotranspiration (correlated with temperature)
        sp_et = np.zeros((rows, cols, n_timesteps))
        for t in range(n_timesteps):
            base_et = max(0, 0.5 + 0.2 * sp_temp[0, 0, t])
            spatial_var = np.random.normal(0, 0.5, size=(rows, cols))
            sp_et[:, :, t] = np.maximum(0, base_et + spatial_var)  # Ensure non-negative

        return sp_prec, sp_temp, sp_et

    except Exception as e:
        logger.error(f"Error preparing meteorological data: {str(e)}")
        raise


# Try to load meteorological data
try:
    sp_prec, sp_temp, sp_et = load_meteorological_data(prec_path, temp_path, et_path, dem_array.shape)
    print(f"Meteorological data prepared with shape: {sp_prec.shape}")

    # Plot a time series at a random point for visualization
    i, j = dem_array.shape[0] // 2, dem_array.shape[1] // 2  # Center point
    time_range = range(sp_prec.shape[2])  # Time steps

    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

    ax1.bar(time_range, sp_prec[i, j, :], color="blue", alpha=0.7)
    ax1.set_ylabel("Precipitation (mm)")
    ax1.set_title("Meteorological Data Time Series at Center Point")

    ax2.plot(time_range, sp_temp[i, j, :], color="red")
    ax2.set_ylabel("Temperature (°C)")

    ax3.plot(time_range, sp_et[i, j, :], color="green")
    ax3.set_ylabel("Pot. Evapotranspiration (mm)")
    ax3.set_xlabel("Time Step (days)")

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Error preparing meteorological data: {str(e)}")
    # Create minimal dummy meteorological data
    n_timesteps = 10
    sp_prec = np.random.random((dem_array.shape[0], dem_array.shape[1], n_timesteps))
    sp_temp = np.random.random((dem_array.shape[0], dem_array.shape[1], n_timesteps)) * 20
    sp_et = np.random.random((dem_array.shape[0], dem_array.shape[1], n_timesteps)) * 5

### 2.4 Prepare HBV Parameters

The HBV model requires parameters for each grid cell. Parameters can be spatially distributed or uniform across the catchment.

In [ ]:
def prepare_hbv_parameters(dem_shape: Tuple[int, int]) -> np.ndarray:
    """
    Prepare spatially distributed HBV parameters.

    Args:
        dem_shape: Shape of the DEM (rows, cols)

    Returns:
        3D array of parameters with shape (rows, cols, n_params)
    """
    # Number of HBV parameters
    n_params = 10  # For the basic HBV model without snow

    # Default parameter values based on literature
    # These are sample values; in a real application, these would be calibrated
    # Based on Bergström (1992) and Beck et al. (2016)
    default_params = {
        "rfcf": 1.0,  # Rainfall correction factor
        "fc": 250.0,  # Field capacity (mm)
        "beta": 3.0,  # Shape coefficient
        "etf": 0.05,  # Total potential evapotranspiration factor
        "lp": 0.3,  # Wilting point as fraction of fc
        "c_flux": 0.01,  # Capillary flux coefficient
        "k": 0.03,  # Upper zone recession coefficient
        "k1": 0.01,  # Lower zone recession coefficient
        "alpha": 0.3,  # Response box parameter
        "perc": 0.8,  # Percolation rate (mm/day)
    }

    # Create a spatially distributed parameter array
    rows, cols = dem_shape
    sp_pars = np.zeros((rows, cols, n_params))

    # Fill the parameter array with default values
    # In a real application, you might vary these based on land cover, soil type, etc.
    param_indices = list(default_params.keys())

    for i, param_name in enumerate(param_indices):
        # Create a bit of spatial variability (±10% of default value)
        base_value = default_params[param_name]
        variation = np.random.uniform(0.9 * base_value, 1.1 * base_value, size=(rows, cols))
        sp_pars[:, :, i] = variation

    return sp_pars


# Prepare HBV parameters
sp_pars = prepare_hbv_parameters(dem_array.shape)
print(f"HBV parameters prepared with shape: {sp_pars.shape}")

# Display parameter statistics
param_names = ["rfcf", "fc", "beta", "etf", "lp", "c_flux", "k", "k1", "alpha", "perc"]

for i, param in enumerate(param_names):
    mean_val = np.mean(sp_pars[:, :, i])
    min_val = np.min(sp_pars[:, :, i])
    max_val = np.max(sp_pars[:, :, i])
    print(f"{param}: mean={mean_val:.4f}, min={min_val:.4f}, max={max_val:.4f}")


### 2.5 Define Additional Model Parameters

Set up additional parameters needed for the HBV distributed model.

In [ ]:
# Optional: Lake parameters (if lakes are present in the catchment)
lakecell = None  # Tuple of lake cell coordinates (row, col)
q_lake = None  # Time series of lake inflow

# Time factor and catchment area
tfac = 24  # Time step in hours (24 hours = daily data)
area = 100  # Catchment area in square kilometers

# Additional parameters for the HBV model
p2 = [tfac, area]

# Initial states (optional)
init_st = None  # If None, default values will be used

# Long-term mean temperature (optional)
ll_temp = None  # If None, calculated from the temperature data

# Initial discharge (optional)
q_0 = None  # If None, a default value will be used

## 3. Initialize the HBV Model

Now we'll create an instance of the HBV conceptual model.

In [ ]:
# Initialize the HBV conceptual model
conceptual_model = HBV()
print("HBV conceptual model initialized.")

## 4. Run the Distributed HBV Model

Now we'll execute the distributed HBV model using the prepared inputs.

In [ ]:
def run_distributed_hbv(
    conceptual_model: HBV,
    lakecell: Optional[Tuple[int, int]],
    q_lake: Optional[np.ndarray],
    DEM: str,
    flow_acc: Dict,
    flow_acc_plan: np.ndarray,
    sp_prec: np.ndarray,
    sp_et: np.ndarray,
    sp_temp: np.ndarray,
    sp_pars: np.ndarray,
    p2: List[float],
    init_st: Optional[List] = None,
    ll_temp: Optional[np.ndarray] = None,
    q_0: Optional[float] = None,
) -> Tuple[np.ndarray, List, np.ndarray, np.ndarray, np.ndarray]:
    """
    Run the distributed HBV model.

    Args:
        conceptual_model: Instance of HBV class
        lakecell: Tuple of lake cell coordinates (optional)
        q_lake: Array of lake inflow (optional)
        DEM: Path to DEM file
        flow_acc: Dictionary mapping cell indices to upstream cells
        flow_acc_plan: 2D array of flow accumulation plan
        sp_prec: 3D array of precipitation (x, y, t)
        sp_et: 3D array of potential evapotranspiration (x, y, t)
        sp_temp: 3D array of temperature (x, y, t)
        sp_pars: 3D array of HBV parameters (x, y, n_params)
        p2: List [tfac, area]
        init_st: Initial states (optional)
        ll_temp: Long-term mean temperature (optional)
        q_0: Initial discharge (optional)

    Returns:
        Tuple containing outlet discharge, states, routed upper zone Q, lower zone Q, and upper zone Q
    """
    try:
        logger.info("Starting distributed HBV model simulation...")

        # Call the Dist_HBV2 function from DistributedRRM
        qout, st, quz_routed, qlz, quz = DistributedRRM.Dist_HBV2(
            conceptual_model=conceptual_model,
            lakecell=lakecell,
            q_lake=q_lake,
            DEM=DEM,
            flow_acc=flow_acc,
            flow_acc_plan=flow_acc_plan,
            sp_prec=sp_prec,
            sp_et=sp_et,
            sp_temp=sp_temp,
            sp_pars=sp_pars,
            p2=p2,
            init_st=init_st,
            ll_temp=ll_temp,
            q_0=q_0,
        )

        logger.info("Distributed HBV model simulation completed successfully")
        return qout, st, quz_routed, qlz, quz

    except Exception as e:
        logger.error(f"Error running distributed HBV model: {str(e)}")
        raise


# Run the distributed HBV model
try:
    print("Starting HBV distributed model simulation...")
    qout, st, quz_routed, qlz, quz = run_distributed_hbv(
        conceptual_model=conceptual_model,
        lakecell=lakecell,
        q_lake=q_lake,
        DEM=DEM,
        flow_acc=flow_acc,
        flow_acc_plan=flow_acc_plan,
        sp_prec=sp_prec,
        sp_et=sp_et,
        sp_temp=sp_temp,
        sp_pars=sp_pars,
        p2=p2,
        init_st=init_st,
        ll_temp=ll_temp,
        q_0=q_0,
    )
    print("HBV distributed model simulation completed.")
    print(f"Simulated discharge shape: {qout.shape}")

except Exception as e:
    print(f"Error running HBV model: {str(e)}")
    print("Using synthetic discharge data for demonstration...")
    # Create synthetic discharge for demonstration
    n_timesteps = sp_prec.shape[2] if hasattr(sp_prec, "shape") else 365

    # Simple synthetic discharge with seasonal pattern
    time = np.arange(n_timesteps)
    base = 20 + 15 * np.sin(2 * np.pi * time / 365)  # Seasonal pattern
    noise = np.random.normal(0, 5, n_timesteps)  # Random variations
    qout = np.maximum(0, base + noise)  # Ensure non-negative

    # Dummy state variables and flow components
    st = [[0, 0, 0, 0, 0] for _ in range(n_timesteps)]  # Dummy states
    quz_routed = qout * 0.7  # Dummy upper zone routed component
    qlz = qout * 0.3  # Dummy lower zone component
    quz = qout * 0.7  # Dummy upper zone component

## 5. Results Analysis & Visualization

Now we'll analyze and visualize the model results.

In [ ]:
# Plot the simulated discharge
plt.figure(figsize=(12, 6))

# Time axis
time_steps = np.arange(len(qout))

# Plot discharge components
plt.plot(time_steps, qout, "b-", linewidth=2, label="Total Discharge")
plt.plot(time_steps, quz_routed, "g-", alpha=0.7, label="Upper Zone (Routed)")
plt.plot(time_steps, qlz, "r-", alpha=0.7, label="Lower Zone")

plt.title("Simulated Discharge at Catchment Outlet")
plt.xlabel("Time Step")
plt.ylabel("Discharge (m³/s)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Stacked area chart of discharge components
plt.figure(figsize=(12, 6))

# Create stacked area chart
plt.fill_between(time_steps, 0, qlz, color="darkblue", alpha=0.7, label="Lower Zone (Baseflow)")
plt.fill_between(time_steps, qlz, qout, color="skyblue", alpha=0.7, label="Upper Zone (Quick Flow)")

# Add total discharge line
plt.plot(time_steps, qout, "k-", linewidth=1, label="Total Discharge")

plt.title("Discharge Components")
plt.xlabel("Time Step")
plt.ylabel("Discharge (m³/s)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

### 5.1 Water Balance Analysis

Let's analyze the water balance components over time.

In [ ]:
# Extract catchment average precipitation and evapotranspiration
# Calculate catchment average for each time step
avg_prec = np.mean(sp_prec, axis=(0, 1))
avg_et = np.mean(sp_et, axis=(0, 1))

# Convert discharge from m³/s to mm/day for water balance comparison
# Q (m³/s) to Q (mm/day) = Q * 86400 / (area_km² * 10^6) * 1000
# Simplified: Q * 86.4 / area_km²
q_mm = qout * 86.4 / area

# Plot water balance components
plt.figure(figsize=(12, 8))

# Subplot 1: Precipitation and Discharge
plt.subplot(3, 1, 1)
plt.bar(time_steps, avg_prec, color="blue", alpha=0.5, label="Precipitation")
plt.plot(time_steps, q_mm, "r-", linewidth=2, label="Discharge")
plt.ylabel("Water Flux (mm/day)")
plt.legend()
plt.title("Water Balance Components")

# Subplot 2: Evapotranspiration
plt.subplot(3, 1, 2)
plt.bar(time_steps, avg_et, color="green", alpha=0.5, label="Pot. Evapotranspiration")
plt.ylabel("ET (mm/day)")
plt.legend()

# Subplot 3: Cumulative balance
plt.subplot(3, 1, 3)
cum_prec = np.cumsum(avg_prec)
cum_q = np.cumsum(q_mm)
cum_et = np.cumsum(avg_et)

# Simplified storage change = P - Q - ET
storage_change = cum_prec - cum_q - cum_et

plt.plot(time_steps, cum_prec, "b-", linewidth=2, label="Cum. Precipitation")
plt.plot(time_steps, cum_q, "r-", linewidth=2, label="Cum. Discharge")
plt.plot(time_steps, cum_et, "g-", linewidth=2, label="Cum. ET")
plt.plot(time_steps, storage_change, "k--", linewidth=1.5, label="Storage Change")

plt.xlabel("Time Step")
plt.ylabel("Cumulative Water (mm)")
plt.legend()

plt.tight_layout()
plt.show()

## 6. Model Evaluation

If observed discharge data is available, we can evaluate the model performance.

In [ ]:
def evaluate_model(sim_q: np.ndarray, obs_q: np.ndarray) -> Dict[str, float]:
    """Evaluate the model performance using common hydrological metrics.

    Args:
        sim_q: Simulated discharge time series
        obs_q: Observed discharge time series

    Returns:
        Dictionary of evaluation metrics
    """
    # Ensure arrays are the same length
    min_len = min(len(sim_q), len(obs_q))
    sim_q = sim_q[:min_len]
    obs_q = obs_q[:min_len]

    # Nash-Sutcliffe Efficiency (NSE)
    # NSE = 1 - ∑(Qobs - Qsim)² / ∑(Qobs - mean(Qobs))²
    mean_obs = np.mean(obs_q)
    nse_numerator = np.sum((obs_q - sim_q) ** 2)
    nse_denominator = np.sum((obs_q - mean_obs) ** 2)
    nse = 1 - (nse_numerator / nse_denominator)

    # Kling-Gupta Efficiency (KGE)
    # KGE = 1 - √[(r-1)² + (α-1)² + (β-1)²]
    # where r is correlation coefficient, α is the ratio of std devs, β is the ratio of means
    r = np.corrcoef(sim_q, obs_q)[0, 1]
    alpha = np.std(sim_q) / np.std(obs_q)
    beta = np.mean(sim_q) / mean_obs
    kge = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

    # Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((sim_q - obs_q) ** 2))

    # Mean Absolute Error (MAE)
    mae = np.mean(np.abs(sim_q - obs_q))

    # Percent Bias (PBIAS)
    pbias = 100 * np.sum(sim_q - obs_q) / np.sum(obs_q)

    # Return metrics dictionary
    metrics = {"NSE": nse, "KGE": kge, "RMSE": rmse, "MAE": mae, "PBIAS": pbias}

    return metrics


# Create synthetic "observed" data for demonstration
observed_q = qout * (1 + 0.2 * np.random.normal(0, 1, len(qout)))  # Add some noise
observed_q = np.maximum(0, observed_q)  # Ensure non-negative

# Evaluate model performance
metrics = evaluate_model(qout, observed_q)

print("Model Evaluation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot observed vs simulated discharge
plt.figure(figsize=(12, 6))
plt.plot(time_steps, observed_q, "k-", label="Observed")
plt.plot(time_steps, qout, "r-", label="Simulated")
plt.fill_between(
    time_steps,
    observed_q,
    qout,
    where=(observed_q > qout),
    interpolate=True,
    color="blue",
    alpha=0.3,
    label="Underestimation",
)
plt.fill_between(
    time_steps,
    observed_q,
    qout,
    where=(observed_q <= qout),
    interpolate=True,
    color="red",
    alpha=0.3,
    label="Overestimation",
)
plt.title("Observed vs Simulated Discharge")
plt.xlabel("Time Step")
plt.ylabel("Discharge (m³/s)")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

## 7. Summary and Conclusions

We have demonstrated the workflow for setting up and running a distributed HBV model for discharge prediction using the Hapi framework. The key steps include:

1. **Data preparation**: Loading and processing DEM, flow direction/accumulation, meteorological data, and parameter fields.

2. **Model initialization**: Creating an instance of the HBV conceptual model.

3. **Model execution**: Running the distributed HBV model using the `Dist_HBV2` function from the `DistributedRRM` class.

4. **Results analysis**: Visualizing and analyzing the simulated discharge and its components.

5. **Model evaluation**: Assessing model performance using common hydrological metrics like NSE and KGE.

This approach allows for spatially distributed simulation of hydrological processes across a catchment, accounting for spatial variability in topography, meteorological forcing, and model parameters.

## 8. Parameter Calibration with Optuna

In hydrological modeling, parameter calibration is crucial for achieving good model performance. Here, we'll use Optuna, a hyperparameter optimization framework, to calibrate the HBV model parameters.

Optuna provides several advantages:
- Efficient parameter space exploration
- Support for various sampling strategies (TPE, CMA-ES, etc.)
- Parallel computing capabilities
- Trial pruning for early stopping of unpromising trials
- Visualization tools for understanding parameter importance

In [ ]:
# Install Optuna if not already installed
# Uncomment the following line to install
# !pip install optuna

In [ ]:
# Import Optuna and additional required libraries
import optuna
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional, Callable
from datetime import datetime
import logging

# Set up separate logger for the calibration process
calib_logger = logging.getLogger("hbv_calibration")
calib_logger.setLevel(logging.INFO)

In [ ]:
def create_objective_function(
    dem_shape: Tuple[int, int],
    conceptual_model: HBV,
    dem_path: str,
    flow_acc: Dict,
    flow_acc_plan: np.ndarray,
    sp_prec: np.ndarray,
    sp_et: np.ndarray,
    sp_temp: np.ndarray,
    p2: List[float],
    observed_q: np.ndarray,
    param_ranges: Optional[Dict[str, Tuple[float, float]]] = None,
    objective_metric: str = "KGE",
    lakecell: Optional[Tuple[int, int]] = None,
    q_lake: Optional[np.ndarray] = None,
    init_st: Optional[List] = None,
    ll_temp: Optional[np.ndarray] = None,
    q_0: Optional[float] = None,
) -> Callable:
    """
    Create an objective function for Optuna parameter optimization.

    Args:
        dem_shape: Shape of the DEM (rows, cols)
        conceptual_model: Instance of HBV class
        dem_path: Path to DEM file
        flow_acc: Dictionary mapping cell indices to upstream cells
        flow_acc_plan: 2D array of flow accumulation plan
        sp_prec: 3D array of precipitation (x, y, t)
        sp_et: 3D array of potential evapotranspiration (x, y, t)
        sp_temp: 3D array of temperature (x, y, t)
        p2: List [tfac, area]
        observed_q: Observed discharge for calibration
        param_ranges: Dictionary with parameter ranges (min, max) for calibration
        objective_metric: Metric to optimize ("KGE", "NSE", "RMSE", "MAE")
        lakecell: Tuple of lake cell coordinates (optional)
        q_lake: Array of lake inflow (optional)
        init_st: Initial states (optional)
        ll_temp: Long-term mean temperature (optional)
        q_0: Initial discharge (optional)

    Returns:
        Objective function for Optuna optimization
    """
    # Default parameter ranges if not provided
    if param_ranges is None:
        param_ranges = {
            "rfcf": (0.5, 2.0),  # Rainfall correction factor
            "fc": (50.0, 500.0),  # Field capacity (mm)
            "beta": (1.0, 6.0),  # Shape coefficient
            "etf": (0.01, 0.5),  # Total potential evapotranspiration factor
            "lp": (0.1, 0.9),  # Wilting point as fraction of fc
            "c_flux": (0.0, 0.1),  # Capillary flux coefficient
            "k": (0.01, 0.5),  # Upper zone recession coefficient
            "k1": (0.001, 0.1),  # Lower zone recession coefficient
            "alpha": (0.1, 0.9),  # Response box parameter
            "perc": (0.1, 1.0),  # Percolation rate (mm/day)
        }

    param_names = list(param_ranges.keys())

    def objective(trial: optuna.Trial) -> float:
        # Generate parameters using Optuna's suggest functions
        trial_params = {}
        for param_name in param_names:
            param_min, param_max = param_ranges[param_name]
            trial_params[param_name] = trial.suggest_float(param_name, param_min, param_max)

        calib_logger.info(f"Trial {trial.number}: {trial_params}")

        # Create parameter array with spatially uniform parameters for this trial
        rows, cols = dem_shape
        sp_pars = np.zeros((rows, cols, len(param_names)))

        # Fill parameter array with trial parameters (uniform across catchment for simplicity)
        for i, param_name in enumerate(param_names):
            sp_pars[:, :, i] = trial_params[param_name]

        try:
            # Run HBV model with trial parameters
            qout, _, _, _, _ = run_distributed_hbv(
                conceptual_model=conceptual_model,
                lakecell=lakecell,
                q_lake=q_lake,
                DEM=dem_path,
                flow_acc=flow_acc,
                flow_acc_plan=flow_acc_plan,
                sp_prec=sp_prec,
                sp_et=sp_et,
                sp_temp=sp_temp,
                sp_pars=sp_pars,
                p2=p2,
                init_st=init_st,
                ll_temp=ll_temp,
                q_0=q_0,
            )

            # Calculate performance metrics
            metrics = evaluate_model(qout, observed_q)

            # Return the objective value (negative because Optuna minimizes)
            if objective_metric == "KGE":
                return -metrics["KGE"]
            elif objective_metric == "NSE":
                return -metrics["NSE"]
            elif objective_metric == "RMSE":
                return metrics["RMSE"]
            elif objective_metric == "MAE":
                return metrics["MAE"]
            else:
                return -metrics["KGE"]  # Default to KGE

        except Exception as e:
            calib_logger.error(f"Error in trial {trial.number}: {str(e)}")
            # Return a poor score for failed trials
            if objective_metric in ["KGE", "NSE"]:
                return -999.0  # For metrics where higher is better (using negative)
            else:
                return 999.0  # For metrics where lower is better

    return objective

In [ ]:
def run_optuna_calibration(
    objective_function: Callable,
    n_trials: int = 100,
    n_jobs: int = 1,
    direction: str = "minimize",
    study_name: str = "hbv_calibration",
    storage: Optional[str] = None,
    sampler: Optional[optuna.samplers.BaseSampler] = None,
) -> optuna.study.Study:
    """
    Run Optuna optimization for parameter calibration.

    Args:
        objective_function: Objective function for optimization
        n_trials: Number of trials to run
        n_jobs: Number of parallel jobs
        direction: Direction of optimization ("minimize" or "maximize")
        study_name: Name of the study
        storage: Database URL for storing study results (optional)
        sampler: Optuna sampler to use (optional)

    Returns:
        Completed Optuna study object
    """
    # Create sampler if not provided
    if sampler is None:
        sampler = optuna.samplers.TPESampler(seed=42)

    # Create study
    study = optuna.create_study(
        direction=direction, study_name=study_name, storage=storage, sampler=sampler, load_if_exists=True
    )

    # Run optimization
    start_time = datetime.now()
    print(f"Starting parameter calibration with {n_trials} trials...")
    study.optimize(objective_function, n_trials=n_trials, n_jobs=n_jobs)
    end_time = datetime.now()

    # Print results
    print(f"\nCalibration completed in {end_time - start_time}")
    print(f"Best trial: #{study.best_trial.number}")
    print(f"Best value: {-study.best_value if direction == 'minimize' else study.best_value}")
    print("\nBest parameters:")
    for param_name, param_value in study.best_params.items():
        print(f"  {param_name}: {param_value:.6f}")

    return study

In [ ]:
# Example: Set up HBV calibration with Optuna
# Note: This is a demonstration; in practice you would use real observed data


def run_calibration_example(
    use_real_optimization: bool = False,  # Set to True to run full optimization (time-consuming)
    n_trials: int = 5,  # Use a small number for demonstration
    n_jobs: int = 1,  # Use 1 for demonstration
):
    # Ensure we have all required data
    if "sp_prec" not in globals() or "observed_q" not in globals():
        print("Error: Required data variables not found.")
        return None

    # Parameter ranges to explore
    param_ranges = {
        "rfcf": (0.7, 1.5),  # Rainfall correction factor
        "fc": (100.0, 400.0),  # Field capacity (mm)
        "beta": (1.5, 5.0),  # Shape coefficient
        "etf": (0.05, 0.3),  # Total potential evapotranspiration factor
        "lp": (0.2, 0.8),  # Wilting point as fraction of fc
        "c_flux": (0.0, 0.05),  # Capillary flux coefficient
        "k": (0.01, 0.2),  # Upper zone recession coefficient
        "k1": (0.005, 0.05),  # Lower zone recession coefficient
        "alpha": (0.2, 0.8),  # Response box parameter
        "perc": (0.3, 0.9),  # Percolation rate (mm/day)
    }

    # Create objective function
    objective_fn = create_objective_function(
        dem_shape=dem_array.shape,
        conceptual_model=conceptual_model,
        dem_path=DEM,
        flow_acc=flow_acc,
        flow_acc_plan=flow_acc_plan,
        sp_prec=sp_prec,
        sp_et=sp_et,
        sp_temp=sp_temp,
        p2=p2,
        observed_q=observed_q,
        param_ranges=param_ranges,
        objective_metric="KGE",  # Optimize for Kling-Gupta Efficiency
    )

    if use_real_optimization:
        # Run optimization
        study = run_optuna_calibration(
            objective_function=objective_fn,
            n_trials=n_trials,
            n_jobs=n_jobs,
            direction="minimize",  # We're minimizing negative KGE
        )
        return study
    else:
        print(
            "Calibration setup is complete. Set use_real_optimization=True to run actual optimization."
        )
        print(
            "Note: Actual calibration can be time-consuming depending on catchment size and number of trials."
        )
        return None


# Set up the calibration (but don't run it by default to avoid long execution)
calibration_setup = run_calibration_example(use_real_optimization=False)

In [ ]:
# Function to visualize Optuna results
def visualize_optuna_results(study: optuna.study.Study):
    """Visualize the results of Optuna parameter calibration."""
    if study is None:
        print("No study to visualize. Run the optimization first.")
        return

    # Importance of each parameter
    try:
        param_importance = optuna.importance.get_param_importances(study)

        # Plot parameter importance
        plt.figure(figsize=(12, 6))
        importance_df = pd.DataFrame(
            list(param_importance.items()), columns=["Parameter", "Importance"]
        ).sort_values("Importance", ascending=False)

        bars = plt.bar(importance_df["Parameter"], importance_df["Importance"], color="skyblue")
        plt.title("Parameter Importance")
        plt.xlabel("Parameter")
        plt.ylabel("Importance Score")
        plt.xticks(rotation=45)
        plt.grid(axis="y", alpha=0.3)
        for bar in bars:
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width() / 2.0,
                height + 0.01,
                f"{height:.3f}",
                ha="center",
                va="bottom",
            )
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Could not calculate parameter importance: {str(e)}")

    # Plot optimization history
    plt.figure(figsize=(10, 6))
    plt.plot(
        [t.number for t in study.trials],
        [-t.value for t in study.trials],  # Convert back from negative
        "o-",
        alpha=0.6,
    )
    plt.axhline(y=-study.best_value, color="r", linestyle="--", label="Best Value")
    plt.title("Optimization History")
    plt.xlabel("Trial Number")
    plt.ylabel("Objective Value (KGE)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Try to generate contour plots for pairs of important parameters
    try:
        # Plot contour plots for parameter combinations
        fig = optuna.visualization.plot_contour(study)
        fig.show()
    except Exception as e:
        print(f"Could not generate contour plots: {str(e)}")

In [ ]:
# Function to apply calibrated parameters and evaluate model performance
def apply_calibrated_parameters(
    study: optuna.study.Study,
    dem_shape: Tuple[int, int],
    conceptual_model: HBV,
    dem_path: str,
    flow_acc: Dict,
    flow_acc_plan: np.ndarray,
    sp_prec: np.ndarray,
    sp_et: np.ndarray,
    sp_temp: np.ndarray,
    p2: List[float],
    observed_q: np.ndarray,
    lakecell: Optional[Tuple[int, int]] = None,
    q_lake: Optional[np.ndarray] = None,
    init_st: Optional[List] = None,
    ll_temp: Optional[np.ndarray] = None,
    q_0: Optional[float] = None,
):
    """Apply calibrated parameters and evaluate model performance."""
    if study is None:
        print("No calibration study available.")
        return

    # Extract best parameters
    best_params = study.best_params
    print("\nRunning model with calibrated parameters:")
    for param, value in best_params.items():
        print(f"  {param}: {value:.6f}")

    # Create parameter array with calibrated values
    param_names = list(best_params.keys())
    rows, cols = dem_shape
    sp_pars_calibrated = np.zeros((rows, cols, len(param_names)))

    for i, param_name in enumerate(param_names):
        sp_pars_calibrated[:, :, i] = best_params[param_name]

    # Run model with calibrated parameters
    qout_calibrated, st_calibrated, quz_routed_calibrated, qlz_calibrated, quz_calibrated = (
        run_distributed_hbv(
            conceptual_model=conceptual_model,
            lakecell=lakecell,
            q_lake=q_lake,
            DEM=dem_path,
            flow_acc=flow_acc,
            flow_acc_plan=flow_acc_plan,
            sp_prec=sp_prec,
            sp_et=sp_et,
            sp_temp=sp_temp,
            sp_pars=sp_pars_calibrated,
            p2=p2,
            init_st=init_st,
            ll_temp=ll_temp,
            q_0=q_0,
        )
    )

    # Calculate performance metrics
    metrics_calibrated = evaluate_model(qout_calibrated, observed_q)

    print("\nPerformance metrics with calibrated parameters:")
    for metric, value in metrics_calibrated.items():
        print(f"  {metric}: {value:.4f}")

    # Plot results
    time_steps = np.arange(len(observed_q))

    plt.figure(figsize=(12, 6))
    plt.plot(time_steps, observed_q, "k-", label="Observed")
    plt.plot(time_steps, qout_calibrated, "r-", label="Calibrated Model")

    plt.title("Calibrated HBV Model Performance")
    plt.xlabel("Time Step")
    plt.ylabel("Discharge (m³/s)")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Return calibrated outputs
    return qout_calibrated, st_calibrated, quz_routed_calibrated, qlz_calibrated, quz_calibrated

### Running a Full Calibration

To run a full calibration process with your real data:

1. Ensure you have observed discharge data for calibration
2. Set appropriate parameter ranges based on literature or expert knowledge
3. Choose an appropriate objective function (KGE, NSE, RMSE, etc.)
4. Run the calibration with a sufficient number of trials (typically 100-1000)
5. Analyze the results and apply the best parameters

```python
# Example of full calibration
study = run_calibration_example(use_real_optimization=True, n_trials=100, n_jobs=4)

# Visualize results
visualize_optuna_results(study)

# Apply calibrated parameters
calibrated_outputs = apply_calibrated_parameters(
    study=study,
    dem_shape=dem_array.shape,
    conceptual_model=conceptual_model,
    dem_path=DEM,
    flow_acc=flow_acc,
    flow_acc_plan=flow_acc_plan,
    sp_prec=sp_prec,
    sp_et=sp_et,
    sp_temp=sp_temp,
    p2=p2,
    observed_q=observed_q
)
```

You can also save the Optuna study to a database for persistence and later analysis:

```python
# Create study with database storage
study = optuna.create_study(
    study_name="hbv_calibration",
    storage="sqlite:///hbv_calibration.db",
    direction="minimize",
    load_if_exists=True
)

# Load existing study
existing_study = optuna.load_study(
    study_name="hbv_calibration",
    storage="sqlite:///hbv_calibration.db"
)
```